In [5]:
# Import all needed libraries
! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [6]:
# GCP Authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [7]:
client = bigquery.Client()

In [8]:
# Exploring raw dataset
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [9]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [10]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [11]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [12]:
# Determine available data tables
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [13]:
# Exploring data in posts_questions
hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-01-01'
        ORDER BY accepted_answer_id
        LIMIT 10"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_questions = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526420,How to check if values in individiual rows of ...,<p>Suppose I have the following data.table:</p...,65526457,2,2,None,2021-01-01 00:05:46.310000+00:00,2.0,2021-01-01 02:15:42.223000+00:00,NaT,None,NaN,None,6335637,None,1,2,r|data.table|rowwise,62
1,65526423,d3.js : Generating axis ticks for ordinal values,<p>I want to use ordinal scale in x-axis with ...,65526533,1,0,None,2021-01-01 00:06:09.007000+00:00,NaN,2021-01-01 16:25:58.447000+00:00,NaT,None,NaN,None,9574155,None,1,2,javascript|d3.js,38
2,65526490,Is there a C macro that replaces varied length...,<p>I want to be able to:</p>\n<pre><code>#defi...,65526541,1,1,None,2021-01-01 00:20:09.553000+00:00,0.0,2021-01-01 00:37:24.277000+00:00,NaT,None,NaN,None,12743240,None,1,2,c|macros,35
3,65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,65526554,2,0,None,2021-01-01 00:05:43.627000+00:00,NaN,2021-01-01 18:19:46.870000+00:00,2021-01-01 05:23:28.560000+00:00,None,65863.0,None,14694500,None,1,3,c++|std|c++20|allocator,178
4,65526523,Navigate from parent to child in react where e...,<p>In a React app with a parent and child elem...,65526577,1,1,None,2021-01-01 00:30:31.933000+00:00,NaN,2021-01-01 00:46:22.813000+00:00,2021-01-01 00:35:53.623000+00:00,None,8690857.0,None,3002584,None,1,1,reactjs|react-router|react-router-dom,33
5,65526451,How do I define a field in a subclass by stric...,<p>I'm trying to implement the UML diagram bel...,65526589,1,0,None,2021-01-01 00:11:17.787000+00:00,NaN,2021-01-01 10:22:51.790000+00:00,2021-01-01 10:22:51.790000+00:00,None,2458991.0,None,14634129,None,1,1,java|inheritance|uml,50
6,65526591,How to install Python 3.5.x on Ubuntu 18.04 LT...,<p>Simple question: How to install Python 3.5....,65526611,2,1,None,2021-01-01 00:49:45.320000+00:00,NaN,2021-01-01 09:48:41.703000+00:00,NaT,None,NaN,None,6305105,None,1,0,python|ubuntu,73
7,65526474,GDB: Displaying incorrect values in struct,<p>I'm trying to implement the <code>malloc</c...,65526637,1,0,None,2021-01-01 00:16:10.673000+00:00,NaN,2021-01-01 01:02:22.447000+00:00,2021-01-01 00:39:32.250000+00:00,None,1233251.0,None,11035194,None,1,1,struct|gdb|memory-address,32
8,65526590,Vue2 + laravel6 - Component implementation,<p>I just started using Vue2 with Laravel6 and...,65526639,1,0,None,2021-01-01 00:49:32.030000+00:00,NaN,2021-01-01 01:09:45.250000+00:00,2021-01-01 01:07:22.133000+00:00,None,11960598.0,None,14605909,None,1,0,laravel|vue.js|vuejs2|laravel-6,37
9,65526522,Mobile menu css,<p>What's the best way to\nachieve going from ...,65526667,2,0,None,2021-01-01 00:30:28.633000+00:00,NaN,2021-01-01 01:46:35.280000+00:00,NaT,None,NaN,None,6787542,None,1,0,css,38


In [14]:
# Extract data for May 2021 for posts with accepted ansers
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions_df.sample(n=15)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count
46594,67750747,Django/Local Library. How to assign current us...,<p>I've started learning Django and im doing a...,67750870,2021-05-29 11:20:57.980000+00:00,NaN,1,django|django-models|django-views,32
37781,67600237,Set days name as per the language localization,<p>I am facing an issue with showing day names...,67666105,2021-05-19 09:11:29.483000+00:00,NaN,1,xamarin|datepicker|localization,47
9953,67427496,Assign Role to the same security id with ARM T...,<p>I have an ARM templates that assigns the &q...,67430417,2021-05-07 00:17:47.540000+00:00,0.0,0,azure|azure-resource-manager,29
9422,67426072,mySQl - How to use MAX() function with other q...,"<p>Good afternoon:\nGiven a certain database, ...",67426097,2021-05-06 21:17:28.683000+00:00,NaN,0,mysql|sql|select|max,24
30725,67604798,Laravel - Eloquent select with count some rela...,<p>I have following query:</p>\n<pre><code>Pro...,67605480,2021-05-19 14:03:40.453000+00:00,NaN,0,laravel|eloquent,39
16999,67490498,Mysql: group by latest query optimization,<p>I've been stuck on optimizing this query. I...,67490588,2021-05-11 16:08:29.443000+00:00,NaN,0,mysql|sql,23
41952,67702431,DRF non-writable nested serializer related field,<p>I'm using Django 3.2 and DRF</p>\n<p>I have...,67702899,2021-05-26 09:46:34.997000+00:00,NaN,0,django|django-rest-framework,22
36551,67655471,Python function unexpectedly returns True when...,<p>Disclaimer: I'm a bit of a newbie so this m...,67655481,2021-05-23 01:41:26.023000+00:00,NaN,-1,python,30
10682,67437075,Kotlin Unresolved reference while trying to us...,<pre><code>fun main() {\nval arr = arrayOf(&qu...,67437104,2021-05-07 14:43:55.793000+00:00,NaN,0,kotlin,27
19071,67507784,Launching pyspark webui on local mode,<p>How can I view the <code>pyspark</code> WEB...,67508236,2021-05-12 16:44:01.590000+00:00,NaN,0,apache-spark|pyspark|apache-spark-sql,16


In [15]:
# Determine data types for posts_questions_df
posts_questions_df.dtypes

q_id                                int64
q_title                            object
q_body                             object
accepted_answer_id                  int64
q_creation_date       datetime64[ns, UTC]
q_favorite_count                  float64
q_score                             int64
q_tags                             object
q_view_count                        int64
dtype: object

In [16]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_favorite_count', 43400],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [17]:
len(posts_questions_df)

47057

In [18]:
# Drop q_favorite_count column because too many null values
posts_questions_df = posts_questions_df.drop(columns='q_favorite_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count'],
      dtype='object')

In [19]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [20]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
q_score_explore_raw = posts_questions_df.groupby('q_score').count()
q_score_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count
q_score,,,,,,,
-9,1,1,1,1,1,1,1
-8,2,2,2,2,2,2,2
-7,2,2,2,2,2,2,2
-6,13,13,13,13,13,13,13
-5,37,37,37,37,37,37,37
-4,152,152,152,152,152,152,152
-3,440,440,440,440,440,440,440
-2,1111,1111,1111,1111,1111,1111,1111
-1,3218,3218,3218,3218,3218,3218,3218


In [21]:
# Bin score for more meaningful analysis
# Establish the bins.
q_score_bins = [-10, -1, 0, 10, 25, 50, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Low Score (0-10)", "Medium Score (10 - 25)", "High Score (25-50)", "Popular Score (50-101)"]

# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)

In [22]:
# explore scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),4976,4976,4976,4976,4976,4976,4976,4976
Zero Score (0),26580,26580,26580,26580,26580,26580,26580,26580
Low Score (0-10),15469,15469,15469,15469,15469,15469,15469,15469
Medium Score (10 - 25),21,21,21,21,21,21,21,21
High Score (25-50),7,7,7,7,7,7,7,7
Popular Score (50-101),4,4,4,4,4,4,4,4


In [23]:
# Bin score for more meaningful analysis (broader bin)

# Establish the bins.
q_score_bins = [-10, -1, 0, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Positive Score (>0)"]
# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)
# Bin scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),4976,4976,4976,4976,4976,4976,4976,4976
Zero Score (0),26580,26580,26580,26580,26580,26580,26580,26580
Positive Score (>0),15501,15501,15501,15501,15501,15501,15501,15501


In [24]:
# Verify whether there are redundant columns for score_tier
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [25]:
# Verify data in q_score_tier column
posts_questions_df.sample(n=10)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count,q_score_tier
30623,67604281,How to get last element in a ForeignKey relation?,<p>To describe my problem I will use basic mod...,67604666,2021-05-19 13:30:13.850000+00:00,0,django,33,Zero Score (0)
29341,67593567,Accessing different tables using the same mode...,<p>Faced with the following problem: I have tw...,67593669,2021-05-18 20:33:30.563000+00:00,0,c#|sql-server|entity-framework|linq|linq-to-en...,38,Zero Score (0)
26188,67565868,sum values depending on other column without p...,<p>I tried to sum the point but I got an error...,67566242,2021-05-17 07:54:54.270000+00:00,0,python|json|csv,52,Zero Score (0)
27229,67571162,Include R script with markdown tags as externa...,<p>suppose I have an .R file <code>tmp.R</code...,67575407,2021-05-17 13:57:37.617000+00:00,2,r|r-markdown|markdown|knitr|knitr-spin,34,Positive Score (>0)
31475,67611059,How do I enable full OData functionality in .N...,<p>I'm having trouble getting OData to work in...,67611702,2021-05-19 21:18:37.040000+00:00,0,c#|asp.net|asp.net-core|odata|webapi,39,Zero Score (0)
27137,67574496,How would I create an object that passes the a...,<p>Error is saying grade isn't defined and I d...,67574685,2021-05-17 17:33:45.390000+00:00,-1,python,22,Negative Score (<0)
14264,67433768,How to execute a rails command with sweet aler...,<p>I have the following <code>sweet alert 2</c...,67467837,2021-05-07 11:03:59.243000+00:00,1,javascript|html|ruby-on-rails|erb|sweetalert2,45,Positive Score (>0)
18576,67503917,Pandas DataFrame to html with JavaScript code ...,<p>I'm trying to save Pandas DataFrame to html...,67504187,2021-05-12 12:41:59.207000+00:00,1,javascript|pandas,12,Positive Score (>0)
38970,67675095,"Breakpoint of number increase in ""position()"" ...",<p>I am increasing a number in an attribute va...,67676761,2021-05-24 15:47:35.370000+00:00,0,json|xml|xslt|xslt-3.0,51,Zero Score (0)
28632,67587460,Is there a way for generating configuration fi...,<p>I am setting up a amplify project for a cer...,67587671,2021-05-18 13:42:29.223000+00:00,1,amazon-web-services|aws-amplify,18,Positive Score (>0)


In [26]:
# Drop score column because data transformed to the score_tier column
posts_questions_df = posts_questions_df.drop(columns='q_score')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [27]:
# Determine length of title
posts_questions_df['q_title_char_count']= posts_questions_df['q_title'].str.len() #character count
posts_questions_df['q_title_word_count']= posts_questions_df['q_title'].str.split().str.len() #word count
posts_questions_df.head()

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,34,Zero Score (0),95,20
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,31,Zero Score (0),44,7
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,49,Positive Score (>0),97,17
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,43,Positive Score (>0),86,15
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,18,Zero Score (0),114,17


In [28]:
# Bin by title length (char count)
q_title_explore_char = posts_questions_df.groupby('q_title_char_count').count()
q_title_explore_char

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_word_count
q_title_char_count,,,,,,,,,
15,25,25,25,25,25,25,25,25,25
16,22,22,22,22,22,22,22,22,22
17,39,39,39,39,39,39,39,39,39
18,44,44,44,44,44,44,44,44,44
19,51,51,51,51,51,51,51,51,51
...,...,...,...,...,...,...,...,...,...
146,17,17,17,17,17,17,17,17,17
147,25,25,25,25,25,25,25,25,25
148,27,27,27,27,27,27,27,27,27


In [29]:
# Bin title length by char count

# Establish the bins.
q_char_count_bins = [0, 50, 100, 150]
q_char_count_group_names = ["Short (0 - 50)", "Medium (50-100)", "Long (100-150)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_char_count_bin'] = pd.cut(posts_questions_df['q_title_char_count'], q_char_count_bins, labels=q_char_count_group_names)

# Bin by char length
q_char_count_explore = posts_questions_df.groupby('q_title_char_count_bin').count()
q_char_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
q_title_char_count_bin,,,,,,,,,,
Short (0 - 50),18552,18552,18552,18552,18552,18552,18552,18552,18552,18552
Medium (50-100),26079,26079,26079,26079,26079,26079,26079,26079,26079,26079
Long (100-150),2426,2426,2426,2426,2426,2426,2426,2426,2426,2426


In [30]:
# Bin by title length (word count)
q_title_explore_word = posts_questions_df.groupby('q_title_word_count').count()
q_title_explore_word

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_char_count_bin
q_title_word_count,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1
2,75,75,75,75,75,75,75,75,75,75
3,573,573,573,573,573,573,573,573,573,573
4,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452
5,2755,2755,2755,2755,2755,2755,2755,2755,2755,2755
6,4072,4072,4072,4072,4072,4072,4072,4072,4072,4072
7,4857,4857,4857,4857,4857,4857,4857,4857,4857,4857
8,5422,5422,5422,5422,5422,5422,5422,5422,5422,5422
9,5283,5283,5283,5283,5283,5283,5283,5283,5283,5283


In [31]:
# Bin title length by word count

# Establish the bins.
q_title_word_count_bins = [0, 10, 20, 30, 40]
q_title_word_count_group_names = ["Short (0 - 10)", "Medium (10-20)", "Long (20-30)", "XL (30+)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_word_count_bin'] = pd.cut(posts_questions_df['q_title_word_count'], q_title_word_count_bins, labels=q_title_word_count_group_names)

# Bin by char length
q_word_count_explore = posts_questions_df.groupby('q_title_word_count_bin').count()
q_word_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count,q_title_char_count_bin
q_title_word_count_bin,,,,,,,,,,,
Short (0 - 10),29266,29266,29266,29266,29266,29266,29266,29266,29266,29266,29266
Medium (10-20),16900,16900,16900,16900,16900,16900,16900,16900,16900,16900,16900
Long (20-30),877,877,877,877,877,877,877,877,877,877,877
XL (30+),14,14,14,14,14,14,14,14,14,14,14


In [32]:
# Clean up posts_questions_df
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count',
       'q_title_word_count', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [33]:
# Drop q_title_char_count and q_title_word_count columns because data transformed into bins
posts_questions_df = posts_questions_df.drop(columns='q_title_char_count')
posts_questions_df = posts_questions_df.drop(columns='q_title_word_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [34]:
# Explore view count
q_views_explore_raw = posts_questions_df.groupby('q_view_count').count()
q_views_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
5,2,2,2,2,2,2,2,2,2
6,7,7,7,7,7,7,7,7,7
7,10,10,10,10,10,10,10,10,10
8,28,28,28,28,28,28,28,28,28
9,56,56,56,56,56,56,56,56,56
...,...,...,...,...,...,...,...,...,...
8221,1,1,1,1,1,1,1,1,1
8939,1,1,1,1,1,1,1,1,1
9996,1,1,1,1,1,1,1,1,1


In [35]:
q_views_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
38,1057,1057,1057,1057,1057,1057,1057,1057,1057
37,1078,1078,1078,1078,1078,1078,1078,1078,1078
19,1085,1085,1085,1085,1085,1085,1085,1085,1085
20,1163,1163,1163,1163,1163,1163,1163,1163,1163
36,1171,1171,1171,1171,1171,1171,1171,1171,1171
35,1217,1217,1217,1217,1217,1217,1217,1217,1217
34,1255,1255,1255,1255,1255,1255,1255,1255,1255
21,1260,1260,1260,1260,1260,1260,1260,1260,1260
33,1275,1275,1275,1275,1275,1275,1275,1275,1275


In [36]:
# Bin view count for more meaningful analysis
# Establish the bins.
q_view_count_bins = [0,  50, 500, 1000, 5000, 16000]
q_view_count_group_names = ["<50", "50-500", "500-1000", "1000-5000", "5000-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)

In [37]:
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<50,39372,39372,39372,39372,39372,39372,39372,39372,39372,39372
50-500,7618,7618,7618,7618,7618,7618,7618,7618,7618,7618
500-1000,36,36,36,36,36,36,36,36,36,36
1000-5000,25,25,25,25,25,25,25,25,25,25
5000-16000,6,6,6,6,6,6,6,6,6,6


In [38]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_view_count_bins = [0, 10, 20, 30, 40, 50, 16000]
q_view_count_group_names = ["<10", "10-20", "20-30","30-40", "40-50", "50-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<10,215,215,215,215,215,215,215,215,215,215
10-20,6462,6462,6462,6462,6462,6462,6462,6462,6462,6462
20-30,14153,14153,14153,14153,14153,14153,14153,14153,14153,14153
30-40,11636,11636,11636,11636,11636,11636,11636,11636,11636,11636
40-50,6906,6906,6906,6906,6906,6906,6906,6906,6906,6906
50-16000,7685,7685,7685,7685,7685,7685,7685,7685,7685,7685


In [39]:
# Determine word count of body
posts_questions_df['q_body_word_count']= posts_questions_df['q_body'].str.split().str.len() #word count
# Group by body word count
q_body_explore_raw = posts_questions_df.groupby('q_body_word_count').count()
q_body_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
6,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1
8,2,2,2,2,2,2,2,2,2,2,2
9,2,2,2,2,2,2,2,2,2,2,2
10,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...
3393,1,1,1,1,1,1,1,1,1,1,1
3472,1,1,1,1,1,1,1,1,1,1,1
4030,1,1,1,1,1,1,1,1,1,1,1


In [40]:
q_body_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
80,239,239,239,239,239,239,239,239,239,239,239
102,240,240,240,240,240,240,240,240,240,240,240
83,242,242,242,242,242,242,242,242,242,242,242
88,242,242,242,242,242,242,242,242,242,242,242
111,244,244,244,244,244,244,244,244,244,244,244
89,244,244,244,244,244,244,244,244,244,244,244
106,246,246,246,246,246,246,246,246,246,246,246
100,247,247,247,247,247,247,247,247,247,247,247
101,248,248,248,248,248,248,248,248,248,248,248


In [41]:
# Bin body word count for more meaningful analysis
# Establish the bins.
q_body_len_bins = [0,  100, 500, 10000]
q_body_len_group_names = ["<100", "100-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<100,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766,14766
100-500,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381,30381
500-10000,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910


In [42]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_body_len_bins = [0, 50, 100, 250, 500, 10000]
q_body_len_group_names = ["<50", "50-100", "100-250", "250-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<50,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389,3389
50-100,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377,11377
100-250,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844,22844
250-500,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537,7537
500-10000,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910,1910


In [43]:
# Create list with all values within q_body
#q_body = posts_questions_df['q_body']
#q_body

In [44]:
#print(q_body[47056])

In [45]:
# Find all unique HTML tags in q_body list
    # Use re library instead of hardcoding things
    # Use functions because it's a repetitive process
    
    # 1. To find all the unique tags in the q_body listand store in their own list
    # 2. Count tags within each item in q_body list (Beautiful Soup)

#test_str = q_body [47056]

#def find_tags(q_body_str):
    
    
#batRegex = re.compile(r'(<[a-z]*>)')
#m1=batRegex.search(html)
#print batRegex.findall(yourstring)




In [46]:
# Count number of each unique tag for every row in q_body list
    # For each row, how many <p>, how many <img tags>, etc
    # Store as separate column in q_body
    
# Group by tags

# Create function to count tags
    # Create list from tags column
    # Count number of "|" + 1
    
q_tags = posts_questions_df['q_tags']
q_tags # Observe the list of tags


0                      javascript|html|node.js|obs
1        z3|smt|formal-methods|satisfiability|cvc4
2                                     python|numpy
3                                           python
4                                 laravel-8|vuejs3
                           ...                    
47052                                python|random
47053                           postgresql|haskell
47054                           html|google-sheets
47055                             javascript|regex
47056      python|dictionary|recursion|defaultdict
Name: q_tags, Length: 47057, dtype: object

In [90]:
q_tags_count = q_tags.str.count('\\|') + 1 # Count the number of | + 1 to get the number of tags in each row

posts_questions_df['q_tags_count'] = q_tags_count # Append the list as a column to the posts_questions_df 
tag_count = posts_questions_df[['q_tags','q_tags_count']]
tag_count

,q_tags,q_tags_count
0,javascript|html|node.js|obs,4
1,z3|smt|formal-methods|satisfiability|cvc4,5
2,python|numpy,2
3,python,1
4,laravel-8|vuejs3,2
...,...,...
47052,python|random,2
47053,postgresql|haskell,2
47054,html|google-sheets,2
47055,javascript|regex,2


In [91]:
# Group by tag count
tag_count_exlore = tag_count.groupby('q_tags_count').count()
tag_count_exlore

,q_tags
q_tags_count,
1,5746
2,12426
3,13295
4,8942
5,6648


In [48]:
# Expand our query beyond May 2021, go back 12 months

# Create our own tags table and categorize and encode it

In [56]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_answers = pd.DataFrame.from_records(results_list, columns=cols)
posts_answers

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67341731,None,<p>This call is causing it:</p>\n<pre><code>so...,None,None,0,None,2021-05-01 00:00:43.057000+00:00,None,2021-05-01 00:00:43.057000+00:00,NaT,None,NaN,None,1980846,67311357,2,0,None,None
1,67341735,None,<p>It looks like you broke your <code>.vimrc</...,None,None,3,None,2021-05-01 00:02:14.853000+00:00,None,2021-05-01 00:02:14.853000+00:00,NaT,None,NaN,None,3251281,67341681,2,1,None,None
2,67341736,None,<p>After extensive reading of the VBA document...,None,None,0,None,2021-05-01 00:02:18.417000+00:00,None,2021-05-01 00:02:18.417000+00:00,NaT,None,NaN,None,10967640,67326302,2,0,None,None
3,67341740,None,"<p>The double bar is the distance measure, and...",None,None,0,None,2021-05-01 00:02:59.713000+00:00,None,2021-05-01 00:02:59.713000+00:00,NaT,None,NaN,None,15804985,44038581,2,0,None,None
4,67341741,None,<p>Long characters continue to be a problem fo...,None,None,0,None,2021-05-01 00:03:03.637000+00:00,None,2021-05-05 19:39:39.557000+00:00,2021-05-05 19:39:39.557000+00:00,None,1104179.0,None,1104179,48127045,2,0,None,None
5,67341744,None,<pre><code>**Do you mean so?**\nprivate var it...,None,None,4,None,2021-05-01 00:03:39.260000+00:00,None,2021-05-01 00:03:39.260000+00:00,NaT,None,NaN,None,14893497,67341386,2,0,None,None
6,67341745,None,<p>The file could be sent to the current worki...,None,None,0,None,2021-05-01 00:03:47.920000+00:00,None,2021-05-01 00:03:47.920000+00:00,NaT,None,NaN,None,12370687,67336278,2,0,None,None
7,67341746,None,<p>First of all this is not gonna be a technic...,None,None,0,None,2021-05-01 00:03:56.097000+00:00,None,2021-05-01 00:03:56.097000+00:00,NaT,None,NaN,None,15125036,67341278,2,-1,None,None
8,67341748,None,<p>Working nginx config:</p>\n<pre><code>upstr...,None,None,0,None,2021-05-01 00:04:10.380000+00:00,None,2021-05-01 00:04:10.380000+00:00,NaT,None,NaN,None,798502,67312468,2,0,None,None
9,67341749,None,<p>I was able to build appbundle by running in...,None,None,2,None,2021-05-01 00:04:10.487000+00:00,None,2021-05-01 03:58:09.633000+00:00,2021-05-01 03:58:09.633000+00:00,None,2628868.0,None,15120078,67224299,2,2,None,None


In [57]:
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, creation_date, parent_id
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = ['a_id', 'a_creation_date', 'q_id']
posts_answers_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_answers_df

,a_id,a_creation_date,q_id
0,67341731,2021-05-01 00:00:43.057000+00:00,67311357
1,67341735,2021-05-01 00:02:14.853000+00:00,67341681
2,67341736,2021-05-01 00:02:18.417000+00:00,67326302
3,67341740,2021-05-01 00:02:59.713000+00:00,44038581
4,67341741,2021-05-01 00:03:03.637000+00:00,48127045
...,...,...,...
169650,67757977,2021-05-30 05:25:20.073000+00:00,67757936
169651,67757978,2021-05-30 05:25:23.510000+00:00,67757497
169652,67757979,2021-05-30 05:25:34.620000+00:00,67757789
169653,67757981,2021-05-30 05:25:46.913000+00:00,67749523


In [58]:
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin', 'q_view_count_bin', 'q_body_word_count',
       'q_body_len_bin', 'q_tags_count'],
      dtype='object')

In [59]:
#post_questions = post_questions.rename(columns={"id":"parent_id"})
#post_questions

In [77]:
duration_df = pd.merge(posts_questions_df, posts_answers_df, left_on=  ['accepted_answer_id'],
                   right_on= ['a_id'], how = 'left', suffixes=['_pq','_pa'])
duration_df

,q_id_pq,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,a_id,a_creation_date,q_id_pa
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,34,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,2021-05-01 00:14:38.340000+00:00,67341742
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,31,Zero Score (0),Short (0 - 50),Short (0 - 10),30-40,47,<50,5,67341857,2021-05-01 00:26:39.053000+00:00,67341817
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,49,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,2021-05-01 00:36:54.910000+00:00,67341895
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,43,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,243,100-250,1,67341961,2021-05-01 00:46:37.893000+00:00,67341936
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,18,Zero Score (0),Long (100-150),Medium (10-20),10-20,57,50-100,2,67341974,2021-05-01 00:48:52.390000+00:00,67341921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47052,67531286,Runs sometimes and sometimes gives an index error,<p>I want to make a program that randomly simu...,67757704,2021-05-14 08:28:20.750000+00:00,python|random,30,Negative Score (<0),Short (0 - 50),Short (0 - 10),20-30,603,500-10000,2,67757704,2021-05-30 04:35:06.557000+00:00,67531286
47053,67757579,"What does ""SELECT 1 + 1"" :: IO [Only Int] mean...",<p>This is a very simple protgresql haskell ex...,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,15,Zero Score (0),Medium (50-100),Medium (10-20),10-20,106,100-250,2,67757716,2021-05-30 04:37:58.840000+00:00,67757579
47054,67757723,Google Sheet IMPORTXML div class (class within...,<p>=IMPORTXML(&quot;https://www.klsescreener.c...,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,14,Zero Score (0),Medium (50-100),Medium (10-20),10-20,54,50-100,2,67757776,2021-05-30 04:48:57.807000+00:00,67757723
47055,67757792,regEx rule negative,"<p>i am struggling with RegEx, i want to targe...",67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,15,Zero Score (0),Short (0 - 50),Short (0 - 10),10-20,130,100-250,2,67757830,2021-05-30 04:57:39.123000+00:00,67757792


In [78]:
duration_df.drop(columns=['q_title', 'q_body', 'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin', 'q_view_count_bin', 'q_body_word_count',
       'q_body_len_bin', 'q_tags_count'], inplace=True)
duration_df

,q_id_pq,accepted_answer_id,q_creation_date,a_id,a_creation_date,q_id_pa
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,67341801,2021-05-01 00:14:38.340000+00:00,67341742
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,67341857,2021-05-01 00:26:39.053000+00:00,67341817
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,67341911,2021-05-01 00:36:54.910000+00:00,67341895
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,67341961,2021-05-01 00:46:37.893000+00:00,67341936
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,67341974,2021-05-01 00:48:52.390000+00:00,67341921
...,...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,67757704,2021-05-30 04:35:06.557000+00:00,67531286
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,67757716,2021-05-30 04:37:58.840000+00:00,67757579
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,67757776,2021-05-30 04:48:57.807000+00:00,67757723
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,67757830,2021-05-30 04:57:39.123000+00:00,67757792


In [79]:
# Compare accepted_answer_id and answer_id to verify identical
duration_df['accepted_answer_id'].equals(duration_df['a_id'])

True

In [80]:
# Drop "answer_id" column because redundant
duration_df = duration_df.drop(columns=['a_id', 'q_id_pa'])
duration_df = duration_df.rename(columns={"q_id_pq":"q_id"})
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,2021-05-30 04:35:06.557000+00:00
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,2021-05-30 04:37:58.840000+00:00
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,2021-05-30 04:48:57.807000+00:00
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,2021-05-30 04:57:39.123000+00:00


In [81]:
# Add column for day of question_creation_date [question_day]
duration_df['q_day'] = duration_df['q_creation_date'].dt.day_name()
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday
...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,2021-05-30 04:35:06.557000+00:00,Friday
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,2021-05-30 04:37:58.840000+00:00,Sunday
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,2021-05-30 04:48:57.807000+00:00,Sunday
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,2021-05-30 04:57:39.123000+00:00,Sunday


In [82]:
# Add column for hour value of question_creation_date [question_time]
duration_df['q_hour'] = duration_df['q_creation_date'].dt.hour
duration_df['q_hour_min'] = duration_df['q_creation_date'].dt.strftime('%H:%M')

duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38
...,...,...,...,...,...,...,...
47052,67531286,67757704,2021-05-14 08:28:20.750000+00:00,2021-05-30 04:35:06.557000+00:00,Friday,8,08:28
47053,67757579,67757716,2021-05-30 04:02:47.097000+00:00,2021-05-30 04:37:58.840000+00:00,Sunday,4,04:02
47054,67757723,67757776,2021-05-30 04:38:50.357000+00:00,2021-05-30 04:48:57.807000+00:00,Sunday,4,04:38
47055,67757792,67757830,2021-05-30 04:50:54.750000+00:00,2021-05-30 04:57:39.123000+00:00,Sunday,4,04:50


In [83]:
# Add column for duration till accepted answer (answer_creation_date - question_creation_date)
duration_df['accepted_answer_duration'] = (duration_df['a_creation_date']-duration_df['q_creation_date'])/np.timedelta64(1,'h')
duration_df = duration_df.sort_values(by='accepted_answer_duration', ascending=False)
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
46975,67342099,67756344,2021-05-01 01:19:14.210000+00:00,2021-05-29 22:57:23.117000+00:00,Saturday,1,01:19,693.635808
45777,67349217,67741551,2021-05-01 17:43:24.997000+00:00,2021-05-28 15:18:48.583000+00:00,Saturday,17,17:43,645.589885
46098,67352708,67744939,2021-05-02 03:20:42.240000+00:00,2021-05-28 20:10:38.163000+00:00,Sunday,3,03:20,640.832201
41532,67349112,67699250,2021-05-01 17:34:03.697000+00:00,2021-05-26 05:57:48.013000+00:00,Saturday,17,17:34,588.395643
46603,67399034,67750961,2021-05-05 09:56:23.113000+00:00,2021-05-29 11:44:36.820000+00:00,Wednesday,9,09:56,577.803807
...,...,...,...,...,...,...,...,...
25782,67563037,67563038,2021-05-17 01:35:04.927000+00:00,2021-05-17 01:35:04.927000+00:00,Monday,1,01:35,0.000000
16408,67485503,67485504,2021-05-11 11:05:40.837000+00:00,2021-05-11 11:05:40.837000+00:00,Tuesday,11,11:05,0.000000
37859,67666770,67666771,2021-05-24 04:42:53.953000+00:00,2021-05-24 04:42:53.953000+00:00,Monday,4,04:42,0.000000
25652,67561998,67561999,2021-05-16 22:14:34.137000+00:00,2021-05-16 22:14:34.137000+00:00,Sunday,22,22:14,0.000000


In [85]:
ML_input_df = pd.merge(posts_questions_df, duration_df, left_on=  ['q_id'],
                   right_on= ['q_id'], how = 'left', suffixes=['_pq','_dur'])
ML_input_df

,q_id,q_title,q_body,accepted_answer_id_pq,q_creation_date_pq,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,...,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id_dur,q_creation_date_dur,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,34,Zero Score (0),Medium (50-100),Medium (10-20),...,217,100-250,4,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03,0.192894
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,31,Zero Score (0),Short (0 - 50),Short (0 - 10),...,47,<50,5,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17,0.149909
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,49,Positive Score (>0),Medium (50-100),Medium (10-20),...,423,250-500,2,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32,0.076868
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,43,Positive Score (>0),Medium (50-100),Medium (10-20),...,243,100-250,1,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41,0.088950
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,18,Zero Score (0),Long (100-150),Medium (10-20),...,57,50-100,2,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47052,67531286,Runs sometimes and sometimes gives an index error,<p>I want to make a program that randomly simu...,67757704,2021-05-14 08:28:20.750000+00:00,python|random,30,Negative Score (<0),Short (0 - 50),Short (0 - 10),...,603,500-10000,2,67757704,2021-05-14 08:28:20.750000+00:00,2021-05-30 04:35:06.557000+00:00,Friday,8,08:28,380.112724
47053,67757579,"What does ""SELECT 1 + 1"" :: IO [Only Int] mean...",<p>This is a very simple protgresql haskell ex...,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,15,Zero Score (0),Medium (50-100),Medium (10-20),...,106,100-250,2,67757716,2021-05-30 04:02:47.097000+00:00,2021-05-30 04:37:58.840000+00:00,Sunday,4,04:02,0.586595
47054,67757723,Google Sheet IMPORTXML div class (class within...,<p>=IMPORTXML(&quot;https://www.klsescreener.c...,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,14,Zero Score (0),Medium (50-100),Medium (10-20),...,54,50-100,2,67757776,2021-05-30 04:38:50.357000+00:00,2021-05-30 04:48:57.807000+00:00,Sunday,4,04:38,0.168736
47055,67757792,regEx rule negative,"<p>i am struggling with RegEx, i want to targe...",67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,15,Zero Score (0),Short (0 - 50),Short (0 - 10),...,130,100-250,2,67757830,2021-05-30 04:50:54.750000+00:00,2021-05-30 04:57:39.123000+00:00,Sunday,4,04:50,0.112326


In [86]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
zero_time = duration_df.groupby('accepted_answer_duration').count()
zero_time 

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min
accepted_answer_duration,,,,,,,
0.000000,172,172,172,172,172,172,172
0.004931,1,1,1,1,1,1,1
0.005020,1,1,1,1,1,1,1
0.006600,1,1,1,1,1,1,1
0.007744,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
577.803807,1,1,1,1,1,1,1
588.395643,1,1,1,1,1,1,1
640.832201,1,1,1,1,1,1,1


In [87]:
# drop data rows where duration is 0, because assumped glitch
indexNames = ML_input_df[ML_input_df['accepted_answer_duration'] == 0.000000 ].index
ML_input_df.drop(indexNames, inplace=True)
ML_input_df

,q_id,q_title,q_body,accepted_answer_id_pq,q_creation_date_pq,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,...,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id_dur,q_creation_date_dur,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,34,Zero Score (0),Medium (50-100),Medium (10-20),...,217,100-250,4,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03,0.192894
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,z3|smt|formal-methods|satisfiability|cvc4,31,Zero Score (0),Short (0 - 50),Short (0 - 10),...,47,<50,5,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17,0.149909
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,49,Positive Score (>0),Medium (50-100),Medium (10-20),...,423,250-500,2,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32,0.076868
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,43,Positive Score (>0),Medium (50-100),Medium (10-20),...,243,100-250,1,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41,0.088950
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,18,Zero Score (0),Long (100-150),Medium (10-20),...,57,50-100,2,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47052,67531286,Runs sometimes and sometimes gives an index error,<p>I want to make a program that randomly simu...,67757704,2021-05-14 08:28:20.750000+00:00,python|random,30,Negative Score (<0),Short (0 - 50),Short (0 - 10),...,603,500-10000,2,67757704,2021-05-14 08:28:20.750000+00:00,2021-05-30 04:35:06.557000+00:00,Friday,8,08:28,380.112724
47053,67757579,"What does ""SELECT 1 + 1"" :: IO [Only Int] mean...",<p>This is a very simple protgresql haskell ex...,67757716,2021-05-30 04:02:47.097000+00:00,postgresql|haskell,15,Zero Score (0),Medium (50-100),Medium (10-20),...,106,100-250,2,67757716,2021-05-30 04:02:47.097000+00:00,2021-05-30 04:37:58.840000+00:00,Sunday,4,04:02,0.586595
47054,67757723,Google Sheet IMPORTXML div class (class within...,<p>=IMPORTXML(&quot;https://www.klsescreener.c...,67757776,2021-05-30 04:38:50.357000+00:00,html|google-sheets,14,Zero Score (0),Medium (50-100),Medium (10-20),...,54,50-100,2,67757776,2021-05-30 04:38:50.357000+00:00,2021-05-30 04:48:57.807000+00:00,Sunday,4,04:38,0.168736
47055,67757792,regEx rule negative,"<p>i am struggling with RegEx, i want to targe...",67757830,2021-05-30 04:50:54.750000+00:00,javascript|regex,15,Zero Score (0),Short (0 - 50),Short (0 - 10),...,130,100-250,2,67757830,2021-05-30 04:50:54.750000+00:00,2021-05-30 04:57:39.123000+00:00,Sunday,4,04:50,0.112326


In [88]:
ML_input_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id_pq',
       'q_creation_date_pq', 'q_tags', 'q_view_count', 'q_score_tier',
       'q_title_char_count_bin', 'q_title_word_count_bin', 'q_view_count_bin',
       'q_body_word_count', 'q_body_len_bin', 'q_tags_count',
       'accepted_answer_id_dur', 'q_creation_date_dur', 'a_creation_date',
       'q_day', 'q_hour', 'q_hour_min', 'accepted_answer_duration'],
      dtype='object')

In [89]:
ML_input_df.drop(columns=['q_title', 'q_body', 'accepted_answer_id_pq',
       'q_creation_date_pq', 'q_tags', 'q_view_count', 'q_creation_date_dur', 'a_creation_date'], inplace=True)
ML_input_df = ML_input_df.rename(columns={"accepted_answer_id_dur":"accepted_answer_id"})
ML_input_df

,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894
1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),30-40,47,<50,5,67341857,Saturday,0,00:17,0.149909
2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076868
3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,243,100-250,1,67341961,Saturday,0,00:41,0.088950
4,67341921,Zero Score (0),Long (100-150),Medium (10-20),10-20,57,50-100,2,67341974,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47052,67531286,Negative Score (<0),Short (0 - 50),Short (0 - 10),20-30,603,500-10000,2,67757704,Friday,8,08:28,380.112724
47053,67757579,Zero Score (0),Medium (50-100),Medium (10-20),10-20,106,100-250,2,67757716,Sunday,4,04:02,0.586595
47054,67757723,Zero Score (0),Medium (50-100),Medium (10-20),10-20,54,50-100,2,67757776,Sunday,4,04:38,0.168736
47055,67757792,Zero Score (0),Short (0 - 50),Short (0 - 10),10-20,130,100-250,2,67757830,Sunday,4,04:50,0.112326


In [93]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        LIMIT 10"""
    )

results = query_job.result()  
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes = pd.DataFrame.from_records(results_list, columns=cols)
votes

,id,creation_date,post_id,vote_type_id
0,14472571,2011-04-25 00:00:00+00:00,5774048,2
1,14473282,2011-04-25 00:00:00+00:00,5774283,2
2,14478256,2011-04-25 00:00:00+00:00,3086484,2
3,14479599,2011-04-25 00:00:00+00:00,151005,5
4,14479762,2011-04-25 00:00:00+00:00,5663770,10
5,14481320,2011-04-25 00:00:00+00:00,267761,5
6,14482412,2011-04-25 00:00:00+00:00,5733809,2
7,14483023,2011-04-25 00:00:00+00:00,5778782,2
8,14483233,2011-04-25 00:00:00+00:00,5778696,2
9,14483799,2011-04-25 00:00:00+00:00,5774515,1


In [94]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes_df = pd.DataFrame.from_records(results_list, columns=cols)
votes_df

,id,creation_date,post_id,vote_type_id
0,231036551,2021-05-02 00:00:00+00:00,53145951,2
1,231036552,2021-05-02 00:00:00+00:00,49928338,2
2,231036553,2021-05-02 00:00:00+00:00,67351126,2
3,231036554,2021-05-02 00:00:00+00:00,5993326,2
4,231036555,2021-05-02 00:00:00+00:00,67351799,2
...,...,...,...,...
1528804,232681752,2021-05-30 00:00:00+00:00,19808279,2
1528805,232681753,2021-05-30 00:00:00+00:00,66633088,2
1528806,232681754,2021-05-30 00:00:00+00:00,57899013,2
1528807,232681755,2021-05-30 00:00:00+00:00,18933902,2


In [96]:
q_id = ML_input_df['q_id']
q_id

0        67341742
1        67341817
2        67341895
3        67341936
4        67341921
           ...   
47052    67531286
47053    67757579
47054    67757723
47055    67757792
47056    67757788
Name: q_id, Length: 46885, dtype: int64

In [101]:
q_votes_df = pd.merge(votes_df, q_id, left_on=  ['post_id'],
                   right_on= ['q_id'], how = 'right', suffixes=['_v','_qid'])
q_votes_df

,id,creation_date,post_id,vote_type_id,q_id
0,NaN,NaT,NaN,NaN,67341742
1,NaN,NaT,NaN,NaN,67341817
2,NaN,NaT,NaN,NaN,67341895
3,231208490.0,2021-05-05 00:00:00+00:00,67341936.0,2.0,67341936
4,NaN,NaT,NaN,NaN,67341921
...,...,...,...,...,...
68045,231764913.0,2021-05-14 00:00:00+00:00,67531286.0,3.0,67531286
68046,NaN,NaT,NaN,NaN,67757579
68047,NaN,NaT,NaN,NaN,67757723
68048,NaN,NaT,NaN,NaN,67757792


In [102]:
votes_explore = q_votes_df.groupby('vote_type_id').count()
votes_explore

,id,creation_date,post_id,q_id
vote_type_id,,,,
2.0,27261,27261,27261,27261
3.0,12053,12053,12053,12053
5.0,3056,3056,3056,3056
8.0,402,402,402,402
9.0,53,53,53,53
10.0,174,174,174,174
11.0,208,208,208,208
15.0,46,46,46,46
16.0,2401,2401,2401,2401
